In [1]:
import pandas as pd
from datetime import datetime
import re

import os
from docx import Document
from docx2pdf import convert

import warnings
warnings.filterwarnings('ignore')

In [2]:
inv_info = pd.read_excel("inv_info_sample.xlsx")


In [3]:
inv_no_list = inv_info['Invoice No.'].unique()


array(['SAMCO_1', 'SAMCO_2', 'SAMCO_3', 'SAMCO_4'], dtype=object)

In [4]:
def get_inv_info(inv_no):
    
    use_info = inv_info[inv_info['Invoice No.'] == inv_no].reset_index(drop = True)

    item_dict = {}
    item_dict['CUSTOMER'] = use_info['Customer'].unique()[0]
    item_dict['CUSTOMER_ADDRESS1'] = use_info['Customer Address1'].unique()[0]
    item_dict['CUSTOMER_ADDRESS2'] = use_info['Customer Address2'].unique()[0]
    item_dict['INV_NO'] = use_info['Invoice No.'].unique()[0]
    item_dict['PAYMENT_TERMS'] = use_info['Payment Terms'].unique()[0]
    
    # products information
    for i in range(1, 16):
        try:
            item_dict['ITEM' + str(i)] = use_info.loc[i-1, 'Item']
            item_dict['DETAIL' + str(i)] = use_info.loc[i-1, 'Detail']
            item_dict['UNITPRICE' + str(i)] = use_info.loc[i-1, 'Unit Price']
            item_dict['QUAN' + str(i)] = use_info.loc[i-1, 'Quantity']
            item_dict['AMT' + str(i)] = use_info.loc[i-1, 'Unit Price'] * use_info.loc[i-1, 'Quantity']
        except:
            item_dict['ITEM' + str(i)] = ""
            item_dict['DETAIL' + str(i)] = ""
            item_dict['UNITPRICE' + str(i)] = ""
            item_dict['QUAN' + str(i)] = ""
            item_dict['AMT' + str(i)] = ""
            
    return item_dict

In [5]:
item_dict = get_inv_info('SAMCO_3')

In [156]:
# def replace_text_in_doc_table(item_dict, inv_no):
#     doc = Document('inv_template.docx')

#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for para in cell.paragraphs:
#                     # 合并所有 run 的文本
#                     full_text = ''.join([run.text for run in para.runs])

#                     # 替换文本
#                     for i in range(1, 16):
#                         for name in ['ITEM', 'DETAIL', 'UNITPRICE', '#', '@']:
#                             full_text = re.sub(r'\b' + re.escape(name + str(i)) + r'\b',
#                                                str(item_dict.get(name + str(i), '')), full_text)

#                     para.clear()
#                     para.add_run(full_text)

#     new_doc_path = inv_no + '.docx'
#     doc.save(new_doc_path)

# # 调用函数
# replace_text_in_doc_table(item_dict, inv_no)

In [ ]:
# def replace_text_in_doc_table(item_dict, inv_no):
#     doc = Document('inv_template.docx')
    
#     inv_no = item_dict['INV_NO']
#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for para in cell.paragraphs:
#                     for run in para.runs:
#                         for i in range(1, 16):
#                             for name in ['ITEM', 'DETAIL', 'UNITPRICE', 'QUAN', 'AMT']:
#                                  run.text = re.sub(r'\b' + name + str(i) + r'\b', 
#                                                    str(item_dict[name + str(i)]), 
#                                                    run.text)

#     new_doc_path = inv_no + '.docx'
#     doc.save(new_doc_path)
    
# replace_text_in_doc_table(item_dict, inv_no)

In [6]:
def replace_text_in_doc_table(item_dict):
    doc = Document('inv_template.docx')
    
    inv_no = item_dict['INV_NO']
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    for run in para.runs:
                            for name in item_dict.keys():
                                 run.text = re.sub(r'\b' + name + r'\b', 
                                                   str(item_dict[name]), 
                                                   run.text)

    new_doc_path = inv_no + '.docx'
    doc.save(new_doc_path)
    
replace_text_in_doc_table(item_dict)

In [9]:
for i in inv_info['Invoice No.'].unique():
    item_dict = get_inv_info(i)
    replace_text_in_doc_table(item_dict)
    convert(i + ".docx")
    os.remove(i + ".docx")
    print(i + " generated successfully!")

  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_1 generated successfully!


  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_2 generated successfully!


  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_3 generated successfully!


  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_4 generated successfully!


In [8]:
for i in inv_info_sample[inv_info_sample['Invoice No.'].isna()].index:
    [customer, amount, doc_date, row_num, description] = rma_data.loc[i, ['Customer', 'Amount', 'Date', 'Row_Number', 'Description']]
    amount = '{0:.2f}'.format(amount)
    inv_no = "LURMA" + doc_date.strftime('%Y%m%d') + str(row_num).zfill(3)
    
    # create the pdf invoice
    replace_text_in_table(customer, amount, doc_date, inv_no)
    convert(inv_no + ".docx")
    os.remove(inv_no + ".docx")
    
    rma_data.loc[i, 'inv_no'] = inv_no
    print(inv_no + " generated successfully!")